In [2]:
!pip install --upgrade pip
!pip install --upgrade jsonpath-ng
!pip install --upgrade pandas
!pip install sagemaker
!pip install boto3
!pip install s3fs

     |████████████████████████████████| 1.5 MB 546 kB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.2.2
    Uninstalling pip-20.2.2:
      Successfully uninstalled pip-20.2.2
     |████████████████████████████████| 10.4 MB 622 kB/s eta 0:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 0.25.3
    Uninstalling pandas-0.25.3:
      Successfully uninstalled pandas-0.25.3
     |████████████████████████████████| 295 kB 307 kB/s eta 0:00:01
     |████████████████████████████████| 97 kB 680 kB/s eta 0:00:01
Processing /Users/jeffrey.magouirk/Library/Caches/pip/wheels/37/42/d8/1609d310cabebc2cf60eca038fa2b0c8503412963734a6fc31/protobuf3_to_dict-0.1.5-cp37-none-any.whl
  Using cached packaging-20.4-py2.py3-none-any.whl (37 kB)
     |████████████████████████████████| 6.6 MB 674 kB/s eta 0:00:01
     |████████████████████████████████| 69 kB 658 kB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-2.5.4-py2.py3-none-any.

In [4]:
!pip install sagemaker

Processing /Users/jeffrey.magouirk/Library/Caches/pip/wheels/4b/57/4d/da97780a5df5e09aa0d22d0bf753d2d7f785180512a5e1f858/sagemaker-2.5.4-py2.py3-none-any.whl
  Using cached importlib_metadata-1.7.0-py2.py3-none-any.whl (31 kB)
  Using cached smdebug_rulesconfig-0.1.5-py2.py3-none-any.whl (6.2 kB)
Processing /Users/jeffrey.magouirk/Library/Caches/pip/wheels/9b/88/83/48b4b8be0d320cfedb69697841775c870cce91554ede0947ad/boto3-1.14.58-py2.py3-none-any.whl
Processing /Users/jeffrey.magouirk/Library/Caches/pip/wheels/37/42/d8/1609d310cabebc2cf60eca038fa2b0c8503412963734a6fc31/protobuf3_to_dict-0.1.5-cp37-none-any.whl
  Using cached packaging-20.4-py2.py3-none-any.whl (37 kB)
  Using cached s3transfer-0.3.3-py2.py3-none-any.whl (69 kB)
  Using cached botocore-1.17.58-py2.py3-none-any.whl (6.6 MB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 0.23
    Uninstalling importlib-metadata-0.23:
      Successfully uninstalled importlib-metadata-0.23
  At

In [3]:
import os
import json
import boto3
from itertools import chain
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp

import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

s3 = boto3.resource('s3')

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from jsonpath_ng import jsonpath, parse

#from dsmatch import local_bucket,s3_ds_bucket
#from dsmatch.util.parallel import get_n_splits

ModuleNotFoundError: No module named 'sagemaker'

<h3> Bringing in the parsed Burning Glass files of job descriptions </h3>

In [ ]:
input_dir_jd = os.path.join(local_bucket, 'data', 'efc', 'jobseeker-applies','job_descriptions')#Calling parsed jd by BG
folders = [x for x in os.listdir(input_dir_jd) if not x.endswith('.csv')]#calling the folders insde the jobseeker-apples
output_dir_jd='bg-jd-skill-names' # output dir 
output_path_jd = os.path.join(local_bucket, 'data', 'efc', 'jobseeker-applies', output_dir_jd)#output path

total_jd=[]#making an object
for folder in folders:
#     already_processed_names = [f.split('/')[-1][:-4] for f in os.listdir(output_path_jd) if f.endswith('.csv')]#
    filelist=[os.path.join(input_dir_jd, folder, x)\
              for x in os.listdir(os.path.join(input_dir_jd, folder)) \
#               if x.endswith(".json") and x[:-5] not in already_processed_names
             ]
    total_jd.extend(filelist)

print("Number of files need to process",len(total_jd))
#print("Number Processed files",len)
############### Output when the notebook can connect to AWS
######Number of files need to process 40605 #############

Defined function to read in the filelist 

In [ ]:
def process_files(filelist):
    records = []
    for filename in filelist:
        with open(filename, 'r') as f:
            d = json.load(f)
            jsonpath_expr = parse(f'$..skillrollup[*].canonskill[*].variant[*]')
            matches = [match.value for match in jsonpath_expr.find(d)]
            if len(matches) > 0:        
                splitted = filename.split('/')
                date_name = splitted[-2]
                job_application_id = splitted[-1][:-5]
                records.append((date_name, job_application_id, matches))
    return records
            
max_workers = mp.cpu_count() - 1

<h4> Looking at the output of date_name, job_application_id, parsed data(variant) </h4>

In [ ]:
df_0 = pd.DataFrame(records)
df_0.info()
df_0.head()

In [ ]:
import re

df_jd = pd.DataFrame(records, columns=['date', 'job_application_id', 'variant'])


df_jd = df_jd.explode('variant')
df_jd.drop_duplicates(inplace=True)

df_jd['variant'] = df_jd['variant'].str.lower()

###Top 20 langugaes spoken in the world, by number of estimated speakers

L = ['arabic','english','chinese','japanese','mandarin','hinidi','spanish','french','bengali','russian',
    'portuguese','urdu','german','indonesian','swahili','marathi','telugu','turkish','cantonese','tamil',
    'western punjabi','punjabi']

df_jd["Required_language"]= df_jd['variant'].str.contains('|'.join(L), flags=re.I)

print('Required Language = \n',df_jd['Required_language'].value_counts())
df_language = pd.DataFrame(df_jd[df_jd['Required_language']==True])
print('Variant = \n',df_language['variant'].value_counts().sort_values(ascending =False))
df_cnts = pd.DataFrame(df_language['job_application_id'].value_counts().sort_values(ascending=False))
print('Counts of Languages = \n',df_cnts['job_application_id'].value_counts().sort_values(ascending=False))


In [ ]:
##Output of the code above
Required Language = 
 False    340605
True      19116
Name: Required_language, dtype: int64
Variant = 
 english             8242
mandarin            3564
chinese             2624
cantonese           1510
french               798
arabic               769
german               738
russian              275
spanish              236
japanese             191
business english      95
indonesian            44
portuguese            30
Name: variant, dtype: int64
Counts of Languages = 
 2    4880
1    3191
3    1687
4     276
Name: job_application_id, dtype: int64